# Urban Traffic Washingtion Convention Center
Simulate the urban traffic from the city of Bonn using [ArcGIS API for Python](https://developers.arcgis.com/python/).

In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

## Required Python modules
You need to install ArcGIS API for Python, please follow the [guide](https://developers.arcgis.com/python/guide/anaconda/).

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from datetime import date, datetime, timedelta
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from geourban.services import aggregate, query, simulations, top
from geourban.types import GridType, VehicleType

## Connect to ArcGIS Online

In [ ]:
gis = GIS()

## Create a map showing the city of Las Vegas

In [ ]:
washington_map = gis.map('Washington, United States', zoomlevel=13)
washington_map

## Connect to geourban services

The host parameter must target the specific host like \"geoprotests.p.rapidapi.com\". Furthermore, the factory directly access `os.environ['x_rapidapi_key']` and uses the specified API key as a header parameter. Otherwise, `georapid.factory.EnvironmentClientFactory.create_client_with_host()` will raise a `ValueError`.

Please, check out the [RapidAPI Account Creation and Management Guide](https://docs.rapidapi.com/docs/account-creation-and-settings).

In [ ]:
client = EnvironmentClientFactory.create_client_with_host('geourban.p.rapidapi.com')

## Request the top 5 accumulated car traffic grid cells
We request these hotspots nearby the Walter E. Washington Convention Center by using the urban region code `USQ7806370`, the simulation date `2024-02-17`, the vehicle type `Car`, and the grid type `agent`. The returned GeoJSON features represents the grid cells with the highest car throughput.

In [ ]:
convention_center_region_code = 'USQ7806370'
simulation_date = date(2024, 2, 17)
vehicle_type = VehicleType.CAR
grid_type = GridType.AGENT
limit = 5
top_traffic_grid_cells = top(client, convention_center_region_code, simulation_date, vehicle_type, grid_type, limit=limit)
top_traffic_grid_cells

## Convert the returned GeoJSON result into a FeatureSet
The FeatureSet offers direct access to a spatially enabled dataframe. We can easily inspect the time frames (`start_time` - `end_time`) and the number of car vehicles `agent_count`.

In [ ]:
top_traffic_fset = FeatureSet.from_geojson(top_traffic_grid_cells)
top_traffic_sdf = top_traffic_fset.sdf
top_traffic_sdf

## Map the traffic grid cells
We see the highest car throughput being nearby the the Carnegie Library at `07:00`, `08:00` and `18:00` o'clock.

In [ ]:
top_traffic_sdf.spatial.plot(washington_map, renderer_type='s', colors='#E80000', alpha=0.3)
washington_map

## Query the simulated agents nearby
We are using the center of the Carnegie Library, request the simulated agents being within a distance of `250 meters`, and specify a `120 seconds` time window starting from `18:00:00`.

In [ ]:
simulation_datetime = datetime.fromisoformat('2024-02-17T18:00:00')
(latitude, longitude) = (38.9026, -77.0229)
(seconds, meters) = (120, 250)
car_positions = query(client, simulation_datetime, vehicle_type, latitude, longitude, seconds, meters)
car_positions_fset = FeatureSet.from_geojson(car_positions)
car_positions_sdf = car_positions_fset.sdf
car_positions_sdf

## Map the car positions nearby

In [ ]:
washington_map = gis.map('Washington, United States', zoomlevel=13)
washington_map.zoom_to_layer(car_positions_sdf)
car_positions_sdf.spatial.plot(washington_map, renderer_type='s', colors='#E80000', marker_size=7, alpha=0.3)
washington_map